# Intro Scalable Deep Learning: Day 5, Tutorial 2

- **Content creators**: Mehdi Cherti
- **Content reviewers**: Jenia Jitsev 
- **Content supervisors**: Mehdi Cherti

## Tutorial Objectives

![StyleGAN2 architecture](stylegan2.png)

The trend of using large data and large models also applies to generative models in general, and Generative Adversarial Networks (GANs) in particular.
With growing model network size and required compute, it becomes hard to train such kinds of models quickly. Therefore, distributed training has been also used in GANs, and was
successful for reducing training time and making models more stable to train with large effective batch sizes (e.g., see [BigGAN](https://arxiv.org/abs/1809.11096), or [BigBiGAN](https://arxiv.org/abs/1907.02544)).
One of the recent architectures that used distributed training for GANs is [StyleGAN2](https://arxiv.org/abs/1912.04958), an improvement of earlier architectures
[StyleGAN](https://arxiv.org/abs/1812.04948) and [ProGAN](https://arxiv.org/abs/1710.10196).

As an example, with StyleGAN2, the authors observe that we can reduce training time from from 70 days with 1 GPU to 10 days with 8 GPUs using distributed training of 1024x1024 resolution images:


| Configuration | Resolution      | Total kimg | 1 GPU   | 2 GPUs  | 4 GPUs  | 8 GPUs | GPU mem |
| :------------ | :-------------: | :--------: | :-----: | :-----: | :-----: | :----: | :-----: |
| `config-f`    | 1024&times;1024 | 25000      | 69d 23h | 36d 4h  | 18d 14h | 9d 18h | 13.3 GB |
| `config-f`    | 1024&times;1024 | 10000      | 27d 23h | 14d 11h | 7d 10h  | 3d 22h | 13.3 GB |
| `config-e`    | 1024&times;1024 | 25000      | 35d 11h | 18d 15h | 9d 15h  | 5d 6h  | 8.6 GB  |
| `config-e`    | 1024&times;1024 | 10000      | 14d 4h  | 7d 11h  | 3d 20h  | 2d 3h  | 8.6 GB  |
| `config-f`    | 256&times;256   | 25000      | 32d 13h | 16d 23h | 8d 21h  | 4d 18h | 6.4 GB  |
| `config-f`    | 256&times;256   | 10000      | 13d 0h  | 6d 19h  | 3d 13h  | 1d 22h | 6.4 GB  |

(Source: https://github.com/NVlabs/stylegan2/blob/master/README.md)


In this tutorial, we will start from a one node/GPU code for training [StyleGAN 2](https://arxiv.org/abs/1912.04958) and modify it to use distributed training with Horovod. In this tutorial, you will learn how to:

1. use Horovod with PyTorch and observe that it is very similar to what we have seen earlier for TensorFlow
2. distribute the training of another GAN architecture, here StyleGAN 2, which includes two optimizers instead of one, as in the usual setup of training supervised image recognition models

## Exercise 1

The first step is to use Horovod on PyTorch model code. In order to do that, you need to fill the **TODO** parts of `stylegan2_multi_node.py`.
The parts that need to modified are very similar to the parts we modify in TensorFlow:

- We need to import Horovod for PyTorch: `import horovod.torch as hvd`
- We need to use `hvd.init()` for Horovod initialization in the beginning
- We need to broadcast the initial weights from rank zero to all the workers (GPUs) with `hvd.broadcast_parameters` for both the generator and the discriminator. We also need to broadcast the optimizers' states of both the generator and the discriminator with `hvd.broadcast_optimizer_state`
- We need to wrap both the generator and the discriminator optimizers with `hvd.DistributedOptimizer`
- Anything to be displayed or any file to be saved in disk (e.g., model checkpoint) should be done on rank zero
- We need to shard data so that different works received different subsets of the data. This is done using PyTorch's `torch.utils.data.distributed.DistributedSampler`

## Exercise 2

<img src="celeba_and_flowers.png" width="50%">

After you modified all the relevant parts, you can now start training. We have provided two datasets, the [Oxford Flowers dataset](https://www.robots.ox.ac.uk/~vgg/data/flowers/) and the [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) dataset.
Flowers is provided with 64x64 resolution and 128x128 resolution and 256x256 resolution, while CelebA is provided with 64x64 resolution
and 128x128 resolution.

- Run `sbatch run.sh --data_path /p/project/training2306/datasets/flowers/flowers_64.npz --image_size 64 --batch_size 32 --name flowers_64`. Observe the results in `results/flowers_64`.
- Run `sbatch run.sh --data_path /p/project/training2306/datasets/celeba/celeba_64.npz --image_size 64 --batch_size 32 --name celeba_64`. Observe the results in `results/celeba_64`.
- Run `sbatch run.sh --data_path /p/project/training2306/datasets/flowers/flowers_128.npz --image_size 128 --batch_size 32 --name flowers_128`. Observe the results in `results/flowers_128`.
- Run `sbatch run.sh --data_path /p/project/training2306/datasets/celeba/celeba_128.npz --image_size 128 --batch_size 32 --name celeba_128`. Observe the results in `results/celeba_128`.
- Show your favorite flowers and/or celebrities

**HINT**: if the training finishes due to timeout, you can resume it by relaunching the sbatch command with the same arguments, it will resume automatically from the last checkpoint.

# Bonus

- Train with more than 2 nodes and/or increase local batch size, leading to a bigger effective batch size and observe the differences with the baseline training
- Train on 256x256 resolution using `/p/project/training2306/datasets/flowers/flowers_256.npz`.
- For fun, train the model with a concatenation of CelebA and Flowers dataset, will the model become confused ? :)
- Construct your own dataset of pictures and train on it. You can use `util/image_folder_to_npz.py` to generate an `npz` file and learn a model on it:
    - `python image_folder_to_npz.py  --image_folder_path <folder> --resolution <resolution> --output_path <dataset_name>.npz`, where `folder` is a folder with pictures inside
    - Run a training `sbatch run.sh --data_path <dataset_name>.npz --image_size <resolution> --batch_size 32 --name <dataset_name>`

# Summary

In this tutorial, we have seen how to use Horovod with PyTorch. As an example, we used the StyleGAN2 architecture.
Training this architecture is quite costly and with distributed training we can reduce training time significantly.